In [1]:
import os
import codecs
import shutil
import linecache
import pandas as pd
import numpy as np


import torch
import torch.utils.data as Data
torch.set_printoptions(precision=15)
# torch.set_default_tensor_type(torch.DoubleTensor)
"""计算四元数转旋转矩阵"""
def q2rot(qMat):
# qMat格式  qw qx qy qz
    w = qMat[0]
    x = qMat[1]
    y = qMat[2]
    z = qMat[3]
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = np.mat([[1-2*y**2-2*z**2,2*x*y-2*w*z,2*x*z+2*w*y],
              [2*x*y+2*w*z,1-2*x**2-2*z**2,2*y*z-2*w*x],
              [2*x*z-2*w*y,2*y*z+2*w*x,1-2*x**2-2*y**2]])
    return rotMat

"""通过四元数，平移矩阵获得奇次矩阵"""
def q2homoRot(qMat,qTrans):
# qMat格式  qw qx qy qz
# qTrans格式 x y z
    qw = qMat[0]
    qx = qMat[1]
    qy = qMat[2]
    qz = qMat[3]
    x = qTrans[0]
    y = qTrans[1]
    z = qTrans[2]
    
    
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = np.mat([
              [1-2*qy**2-2*qz**2,  2*qx*qy-2*qw*qz,   2*qx*qz+2*qw*qy,  x],
              [2*qx*qy+2*qw*qz,    1-2*qx**2-2*qz**2, 2*qy*qz-2*qw*qx,  y],
              [2*qx*qz-2*qw*qy,    2*qy*qz+2*qw*qx,   1-2*qx**2-2*qy**2,z],
              [0,                   0,                  0,                 1]
    ])
    return rotMat



In [3]:
bPath = "/home/vr717/Documents/qys/code/pointnerf/data_src"
dirPath = bPath + "/guanZi169_scannetAll"
filePath = dirPath + "/sparse/0/images.txt"
imgPath = dirPath + "/images"
posePath = dirPath + "/pose"

# bPath = "/home/vr717/Documents/qys/code/pointnerfOri/pointnerf/data_src"
# dirPath = bPath + "/TanksAndTemple/truckForColmap"
# filePath = dirPath + "/sparse/0/images.txt"
# imgPath = dirPath + "/images"
# posePath = dirPath + "/pose"

In [ ]:
# 把数据集计算完之后写到Log文件中



In [3]:
"""
修改自己数据集的images的name
"""

# 每隔十个选一个测试集
imgListOri = os.listdir(imgPath)

counter = 0
trainCounter = 0
valCounter = 0

for img in imgListOri:
    counter = counter+1
    
    if (counter%10) ==0:
        
        newName =str(f'1_{int(valCounter):04}_{int(valCounter):08}.jpg')
        
        src =  os.path.join(imgPath,img)
        dst = os.path.join(imgPath,newName)
        try:
            os.rename(src,dst)
            valCounter +=1
        except:
            print("error")
    else:
        
        newName =str(f'0_{int(trainCounter):04}_{int(trainCounter):08}.jpg')
        
        src =  os.path.join(imgPath,img)
        dst = os.path.join(imgPath,newName)
        try:
            os.rename(src,dst)
            trainCounter +=1
        except:
            print("error")
    
    

In [4]:
"""
读取colmap 进行旋转矩阵 输出最大值最小值
""" 



imgNum = len(os.listdir(imgPath))
print(imgNum)

df = pd.read_table(filePath)

rawDf = []
nameList = []
Df = []
qMat = []

for i in range(imgNum):
    line = df.loc[i*2 + 3].values[0]
    rawDf.append(line)
print(len(rawDf))


postList = [] #存有每个图片的位姿信息

xList = []
yList = []
zList = []

for i in rawDf:
    line = i.split(' ')
    Df.append(line)
    nameList.append(line[-1])
        
    qMat = []
    qTrans = []
    for i in line[1:5]:
        qMat.append(float(i))
    for i in line[5:8]:
        qTrans.append(float(i))
    
    w2per = q2homoRot(qMat,qTrans)
    #转置 计算成相机到世界的

    testMat = torch.tensor(w2per)

    tra = testMat[:3,:3]
    pos = testMat[:3,3].reshape(3,1)
    
    invMat = torch.transpose(tra,0,1)
#     print((-1*invMat).shape)
#     print((-1*invMat))
    
#     print(pos.shape)
#     print(pos)

# # 左手席右手系转换
#     le2ri = torch.tensor([[1,0,0],[0,1,0],[0,0,-1]])
#     invMat = torch.mm(le2ri.to(torch.float64),invMat)
    
    finalPos =torch.mm((-1*invMat),pos.to(torch.float64))
    print("finalpos",finalPos)
    
    
    
    tem = torch.cat((invMat,finalPos),1)
    finalMat = torch.cat((tem,torch.tensor([[0,0,0,1]])),0)
#     print(finalMat)
    postList.append(finalMat)
    
    
    

os.makedirs(posePath, exist_ok=True)
for ind in range(len(postList)):
    txtName = nameList[ind].split('.')[0] + ".txt"
    with codecs.open( os.path.join(posePath,txtName), 'w','gb18030') as f:
        temList = postList[ind].tolist()
        xList.append(temList[0][-1])
        yList.append(temList[1][-1])
        zList.append(temList[2][-1])
        for item in temList:
#             print(item)
            temLine  = ' '.join(str(i) for i in item)
            temLine = temLine + "\n"
            f.write(temLine)


bbox = [min(xList),min(yList),min(zList),max(xList),max(yList),max(zList)]
print(bbox)        
            
    
    
    
    
    
# print("*******")
# # 写入文件 位姿信息
# with codecs.open(f"./test.txt", 'w','gb18030') as f:
# #                 for da in postList[0]:
# #                 [int(x) for item in list_1 for x in item]
#                 temList = postList[0].tolist()
#                 for item in temList:
#                     print(item)
#                     temLine  = ' '.join(str(i) for i in item)
#                     temLine = temLine + "\n"
#                     f.write(temLine)
# #                 list_2 = [int(x) for item in postList[0] for x in item]
# #                 temlist = ' '.join(str(i) for i in list_2)
# #                 f.write(temlist)

# nameList
# test = pd.DataFrame(rawDf)
# print(test.iloc[:,-1])






60
60
finalpos tensor([[ 0.028687382947881],
        [-2.546916594320991],
        [ 2.120973080525173]], dtype=torch.float64)
finalpos tensor([[ 1.260950737351597],
        [ 3.005523234319030],
        [-2.819268385447171]], dtype=torch.float64)
finalpos tensor([[-2.251139160680842],
        [-1.837940183746875],
        [ 3.272798710875968]], dtype=torch.float64)
finalpos tensor([[ 1.964810247316686],
        [ 2.836515077144254],
        [-2.511401429213246]], dtype=torch.float64)
finalpos tensor([[ 2.098175785832359],
        [ 2.913323634219819],
        [-2.612637417457224]], dtype=torch.float64)
finalpos tensor([[-1.953956133841934],
        [ 0.404962341160665],
        [-1.084019354170492]], dtype=torch.float64)
finalpos tensor([[ 3.781559129512096],
        [ 1.791647377711796],
        [-1.098916790178817]], dtype=torch.float64)
finalpos tensor([[-2.835780245148686],
        [ 2.323959553139239],
        [-2.298353843318774]], dtype=torch.float64)
finalpos tensor([[-4.08387

In [47]:
# /**
#     专门用于计算原本位姿和现在的位姿 truck
# **/

"""
读取colmap 进行旋转矩阵 输出最大值最小值
""" 


import os
import codecs
import shutil
import linecache
from tokenize import Double, Double3

basepath = "/home/vr717/Documents/qys/code/pointnerf/data_src/TanksAndTemple/Truck_oriImg_ownPose"
imgPath = basepath + "/images"
logPath = basepath + "/Truck_COLMAP_SfM.log"
def getNameList(path):
    # os.listdir()方法获取文件夹名字，返回数组
    file_name_list = os.listdir(path)

    returnList = []
    print(len(file_name_list))
    for item in file_name_list:
        temList = item.split('.')
        # print(type(fuck))
        
        
        fuck = temList[0].split('_')
#         print(fuck[2])
        returnList.append(int(fuck[2]))
    
    nameList = [x.split('.')[0] for x in file_name_list]
    return nameList,returnList   

# def setTrainPos(path):
    
#     file_name_list = os.listdir(path)
#     with open("./Truck_COLMAP_SfM.log", 'r') as fp:
#         lines = fp.readlines()
#         # last_line = lines[-1]
    
#         for item in file_name_list:
#             name = item.split('.')
            
#             temList = name[0].split('_')
#             var = temList[2].lstrip('0')
#             ind = int(var)
#             data = []

#             for i in range(1,5):
#                 data.append(lines[(ind-1)*5+i])
#                 # print(data)
#             #将读取数据保存新文件中
#             # print(lines)
#             with codecs.open(f"./pose/{name[0]}.txt", 'w','gb18030') as f:
#                 for da in data:
#                     f.write(da)

        
# # 修改图片名称

# def renameTrain():
#     # path = "/home/vr717/Documents/qys/code/pointnerf/data_src/TanksAndTemple/Truck/imgs"
#     # basepath = "/home/vr717/Documents/qys/code/pointnerf/data_src/TanksAndTemple/Truck"
#     # imgPath = basepath + "/imgs"
    
#     file_name_list = os.listdir(path)
#     returnList = []
#     print(len(file_name_list))
#     counter = 0;
#     for item in file_name_list:
#         temList = item.split('.')
#         # print(type(fuck))
#         fuck = temList[0].lstrip('0')
#         src =  os.path.join(imgPath,item)
#         print(fuck)
#         newName =str(f'0_{int(counter):04}_{int(fuck):08}.jpg')
#         dst = os.path.join(imgPath,newName)
#         counter = counter + 1
#         try:
#             # shutil.copy(src,dst)
#             os.rename(src, dst)
#             # print ('converting %s to %s ...' % (item, newName))
#         except:
#             print ('erroraaa') 
#             print ('converting %s to %s ...' % (src, dst))
#             continue
        



fullName,indName =  getNameList(imgPath)

 
with open(logPath, 'r') as fp:
    lines = fp.readlines()
    # last_line = lines[-1]
    
    for i in range(len(indName)):
        data = []
        for ind in range(1,5):
            data.append(lines[(indName[i]-1)*5+ind])
            # print(data)
        #将读取数据保存新文件中
        with codecs.open(basepath+"/pose/"+fullName[i]+".txt", 'w','gb18030') as f:
            for da in data:
                f.write(da)
    
#     for item in file_name_list:
#         name = item.split('.')

#         temList = name[0].split('_')
#         var = temList[2].lstrip('0')
#         ind = int(var)
#         data = []

#         for i in range(1,5):
#             data.append(lines[(ind-1)*5+i])
#             # print(data)
#         #将读取数据保存新文件中
#         # print(lines)
#         with codecs.open(f"./pose/{name[0]}.txt", 'w','gb18030') as f:
#             for da in data:
#                 f.write(da)



240


In [5]:
# 0.99919874556140409 0.0033690331097773383 0.039609401771154497 -0.0046488467121345465 -1.227935238366261 -0.46258417578522382 3.9303967943567

#   找第90张
# 83 -0.12425159787662125 -0.055501884411909404 0.9833460557734397 0.1204641683101355 0.38664304350701489 -0.49918736954556303 3.3939974218075579 1 

line = [-0.12425159787662125,-0.055501884411909404,0.9833460557734397,0.1204641683101355,0.38664304350701489,-0.49918736954556303,3.3939974218075579]

qMat = []
qTrans = []
for i in line[0:4]:
    qMat.append(float(i))
for i in line[4:7]:
    qTrans.append(float(i))

w2per = q2homoRot(qMat,qTrans)

print("w2per",w2per)

#转置 计算成相机到世界的

testMat = torch.tensor(w2per)

tra = testMat[:3,:3]
pos = testMat[:3,3].reshape(3,1)

invMat = torch.transpose(tra,0,1)
#     print((-1*invMat).shape)
#     print((-1*invMat))

#     print(pos.shape)
#     print(pos)

finalPos =torch.mm((-1*invMat),pos.to(torch.float64))
print("finalpos",finalPos)
tem = torch.cat((invMat,finalPos),1)
finalMat = torch.cat((tem,torch.tensor([[0,0,0,1]])),0)

print("finalMat")
print(finalMat)


w2per [[-0.96296216 -0.07921939 -0.25773661  0.38664304]
 [-0.13909085  0.96481585  0.22312353 -0.49918737]
 [ 0.23099266  0.25070833 -0.94009985  3.39399742]
 [ 0.          0.          0.          1.        ]]
finalpos tensor([[-0.481098268645079],
        [-0.338649898002017],
        [ 3.401728983139672]], dtype=torch.float64)
finalMat
tensor([[-0.962962162503667, -0.139090849047714,  0.230992660700394,
         -0.481098268645079],
        [-0.079219387450064,  0.964815849960149,  0.250708325186286,
         -0.338649898002017],
        [-0.257736614081698,  0.223123533892913, -0.940099849156907,
          3.401728983139672],
        [ 0.000000000000000,  0.000000000000000,  0.000000000000000,
          1.000000000000000]], dtype=torch.float64)


In [6]:

def computeDis(vec1,vec2):
    
    temSum = 0;
    for i in range(3):
        temSum = temSum + (vec1[i]-vec2[i])**2
    print(pow(temSum,0.5))

In [3]:

#  90 和 94
# -0.025848994209229089 -0.050048370620630674 0.98890541646295249 0.13745205486578174 -0.17104078381996965 -0.64183031694857873 2.9903614494473065 1 000092.jpg
lines = [
    [0.99831090744517914, 0.056895381319965521, -0.0091276539187672059,-0.007411720068788724, -0.63373442595332019, -0.69215382923706681, 3.2038694360149176],
    [0.24080063741881738, 0.012590865818623089, 0.96043412153176055,0.13936578243652289,0.0061156485371171724, -1.02526647973362,5.0016120898418821]
]

for line in lines:
    qMat = []
    qTrans = []
    for i in line[0:4]:
        qMat.append(float(i))
    for i in line[4:7]:
        qTrans.append(float(i))

    w2per = q2homoRot(qMat,qTrans)

    print("w2per",w2per)

    #转置 计算成相机到世界的

    testMat = torch.tensor(w2per)

    tra = testMat[:3,:3]
    pos = testMat[:3,3].reshape(3,1)

    invMat = torch.transpose(tra,0,1)
    #     print((-1*invMat).shape)
    #     print((-1*invMat))

    #     print(pos.shape)
    #     print(pos)

#     colmap给的xyz是经过 左乘-Rt  因此同时左乘-R
    finalPos =torch.mm((-1*invMat),pos.to(torch.float64))
    print("finalpos",finalPos)
    tem = torch.cat((invMat,finalPos),1)
    finalMat = torch.cat((tem,torch.tensor([[0,0,0,1]])),0)

    print("finalMat")
    print(finalMat)


print("得出结论，旋转矩阵转置，pos左乘（-1*转置)")

w2per [[ 0.9997235   0.01375976 -0.01906786 -0.63373443]
 [-0.01583704  0.99341596 -0.11346326 -0.69215383]
 [ 0.01738109  0.11373386  0.9933592   3.20386944]
 [ 0.          0.          0.          1.        ]]
finalpos tensor([[ 0.566910794734798],
        [ 0.331928249962169],
        [-3.273211175082762]], dtype=torch.float64)
finalMat
tensor([[ 0.999723504679122, -0.015837044675734,  0.017381087653879,
          0.566910794734798],
        [ 0.013759759274674,  0.993415963980155,  0.113733862741410,
          0.331928249962169],
        [-0.019067858212081, -0.113463256278487,  0.993359203036790,
         -3.273211175082762],
        [ 0.000000000000000,  0.000000000000000,  0.000000000000000,
          1.000000000000000]], dtype=torch.float64)
w2per [[-0.88371305 -0.04293334  0.46605577  0.00611565]
 [ 0.09130413  0.9608373   0.26163953 -1.02526648]
 [-0.45903683  0.27376708 -0.84518446  5.00161209]
 [ 0.          0.          0.          1.        ]]
finalpos tensor([[ 2.394939681

In [11]:
computeDis([0.38664304350701489,-0.49918736954556303,3.3939974218075579],[-0.17104078381996965,-0.64183031694857873,2.9903614494473065])

computeDis([-0.1743129619316871,0.014542232934796537, 1.2715754766660312],[-0.3007782532204113,-0.020116227133387555,1.121577082402207])

computeDis([0.38664304,-0.49918737,3.39399742],[-0.17104078,-0.64183032,2.99036145])

computeDis([ -0.481098268645079, -0.338649898002017,3.401728983139672],[-0.349786149450568,-0.222032068345897, 3.035097051842536])

computeDis([0.230992660700394,0.250708325186286,-0.940099849156907],[0.037365917998272,0.274441563206177,-0.960877524223091])

NameError: name 'computeDis' is not defined

In [49]:
"""
计算bbox
"""

a = "/home/vr717/Documents/qys/code/pointnerf/data_src/TanksAndTemple/Truck_oriImg_ownPose/pose"

def computeOriBbox():
    # maxNum 是最大序号
    
    box = []
    
    file_name_list = os.listdir(a)
    
    xMin = 10000
    xMax = -10000
    yMin = 10000
    yMax = -10000
    zMin = 10000
    zMax = -10000
    
    for i in file_name_list:
        filePath = os.path.join(posePath,i)
        
        with open(filePath,'r') as fp:
        
            lines = fp.readlines()
            linex = lines[0]
            liney = lines[1]
            linez = lines[2]
            
            xtem = float(linex.split(' ')[3].rstrip())
            ytem = float(liney.split(' ')[3].rstrip())
            ztem = float(linez.split(' ')[3].rstrip())
            
            xMin = min(xMin,xtem)
            xMax = max(xMax,xtem)
            
            yMin = min(yMin,ytem)
            yMax = max(yMax,ytem)
            
            zMin = min(zMin,ztem)
            zMax = max(zMax,ztem)
    box.append(xMin)
    box.append(yMin)
    box.append(zMin)
    
    box.append(xMax)
    box.append(yMax)
    box.append(zMax)
    return box  

computeOriBbox()

[-3.776902245537,
 -0.461799733737,
 -4.702693142822,
 4.987772425471,
 0.483214295749,
 4.49278507721]

In [ ]:
"""
处理训练集
"""

In [10]:
"""
对视频抽帧
"""

import os.path
import time
import cv2
 
video_path = "./data_src/guanZi169/video/guanZi169.mp4"
save_path = video_path[:video_path.rfind('.')]
os.makedirs(save_path, exist_ok=True)
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print('FPS:{:.2f}'.format(fps))
rate = cap.get(5)
frame_num = cap.get(7)
duration = frame_num/rate
print('video total time:{:.2f}s'.format(duration))
 
# width, height = 1920, 1080
# interval = int(fps) * 4
interval = 3
process_num = frame_num // interval
print('process frame:{:.0f}'.format(process_num))
 
cnt = 0
num = 0
 
t0 = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        cnt += 1
        if cnt % interval == 0:
            num += 1
            # frame = cv.resize(frame, (width, height))
            cv2.imwrite(save_path + "/%d.jpg" % (num-1), frame)
            remain_frame = process_num - num
            t1 = time.time() - t0
            t0 = time.time()
#             print("Processing %07d.jpg, remain frame: %d, remain time: %.2fs" % (num, remain_frame, remain_frame * t1))
            print("Processing %d.jpg, remain frame: %d, remain time: %.2fs" % (num-1, remain_frame, remain_frame * t1))
    else:
        break
    if cv2.waitKey(1) & 0xff == 27:
        break
 
cap.release()
cv2.destroyAllWindows()
print("done")




FPS:29.00
video total time:36.90s
process frame:369
Processing 0.jpg, remain frame: 368, remain time: 19.63s
Processing 1.jpg, remain frame: 367, remain time: 7.97s
Processing 2.jpg, remain frame: 366, remain time: 23.18s
Processing 3.jpg, remain frame: 365, remain time: 7.95s
Processing 4.jpg, remain frame: 364, remain time: 7.94s
Processing 5.jpg, remain frame: 363, remain time: 7.96s
Processing 6.jpg, remain frame: 362, remain time: 7.95s
Processing 7.jpg, remain frame: 361, remain time: 8.04s
Processing 8.jpg, remain frame: 360, remain time: 7.94s
Processing 9.jpg, remain frame: 359, remain time: 22.28s
Processing 10.jpg, remain frame: 358, remain time: 7.84s
Processing 11.jpg, remain frame: 357, remain time: 7.80s
Processing 12.jpg, remain frame: 356, remain time: 7.77s
Processing 13.jpg, remain frame: 355, remain time: 7.75s
Processing 14.jpg, remain frame: 354, remain time: 7.30s
Processing 15.jpg, remain frame: 353, remain time: 7.51s
Processing 16.jpg, remain frame: 352, remai

Processing 152.jpg, remain frame: 216, remain time: 4.59s
Processing 153.jpg, remain frame: 215, remain time: 4.48s
Processing 154.jpg, remain frame: 214, remain time: 4.59s
Processing 155.jpg, remain frame: 213, remain time: 4.67s
Processing 156.jpg, remain frame: 212, remain time: 4.57s
Processing 157.jpg, remain frame: 211, remain time: 4.57s
Processing 158.jpg, remain frame: 210, remain time: 4.56s
Processing 159.jpg, remain frame: 209, remain time: 4.51s
Processing 160.jpg, remain frame: 208, remain time: 4.42s
Processing 161.jpg, remain frame: 207, remain time: 4.40s
Processing 162.jpg, remain frame: 206, remain time: 4.47s
Processing 163.jpg, remain frame: 205, remain time: 4.44s
Processing 164.jpg, remain frame: 204, remain time: 4.20s
Processing 165.jpg, remain frame: 203, remain time: 4.51s
Processing 166.jpg, remain frame: 202, remain time: 4.49s
Processing 167.jpg, remain frame: 201, remain time: 4.44s
Processing 168.jpg, remain frame: 200, remain time: 4.46s
Processing 169

Processing 298.jpg, remain frame: 70, remain time: 1.53s
Processing 299.jpg, remain frame: 69, remain time: 1.51s
Processing 300.jpg, remain frame: 68, remain time: 1.45s
Processing 301.jpg, remain frame: 67, remain time: 1.58s
Processing 302.jpg, remain frame: 66, remain time: 1.39s
Processing 303.jpg, remain frame: 65, remain time: 1.30s
Processing 304.jpg, remain frame: 64, remain time: 1.37s
Processing 305.jpg, remain frame: 63, remain time: 1.36s
Processing 306.jpg, remain frame: 62, remain time: 1.31s
Processing 307.jpg, remain frame: 61, remain time: 1.49s
Processing 308.jpg, remain frame: 60, remain time: 1.31s
Processing 309.jpg, remain frame: 59, remain time: 1.30s
Processing 310.jpg, remain frame: 58, remain time: 1.26s
Processing 311.jpg, remain frame: 57, remain time: 1.22s
Processing 312.jpg, remain frame: 56, remain time: 1.32s
Processing 313.jpg, remain frame: 55, remain time: 1.07s
Processing 314.jpg, remain frame: 54, remain time: 1.08s
Processing 315.jpg, remain fram

'./data_src/guanZi169/video/guanZi169'

In [ ]:


camrotc2w = [[-0.96296216,-0.07921939,-0.25773661,0.38664304]
 [-0.13909085,0.96481585,0.22312353,-0.49918737],
 [ 0.23099266,0.25070833,-0.94009985,3.39399742],
]

campos = [1,2,1]

